In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score,recall_score,accuracy_score,confusion_matrix
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report

# Reading the file and checking the missing values

In [ ]:
data=pd.read_csv("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
data.head(3)

There arent any missing values! :)

In [ ]:
data.isnull().sum()

# EDA 

In [ ]:
data.describe()

covert "quality" into classes(0=bad,1=good) by writing a custom transformer!

In [ ]:
class quality_mark(BaseEstimator,TransformerMixin):
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        X["quality_mark"]=pd.cut(X["quality"],bins=[2, 6.5, 8. ],labels=[0,1])
        X.drop("quality",axis=1,inplace=True)
        return X

In [ ]:
qm=quality_mark()
qm.fit_transform(data)


In [ ]:
sns.countplot(data["quality_mark"])

In [ ]:
data["quality_mark"]=data["quality_mark"].astype(int)
core=data.corr()
trimat=core.where(np.triu(np.ones(core.shape),k=1).astype(bool))
sns.heatmap(trimat,cmap="Reds")

search the columns which have pearsons correlation more than 0.75.there are none!!:) 

In [ ]:
pear_col=[column for column in trimat.columns if any(trimat[column] > 0.75)]
pear_col

In [ ]:
plt.figure(figsize=(2,2))
sns.distplot(data["chlorides"])
plt.figure(figsize=(2,2))
sns.distplot(data["sulphates"])
plt.figure(figsize=(2,2))
sns.distplot(data["density"])


****THE DATA IS SKEWED! All the variables are not normally distributed


#  Handling skewed data

Almost all models expect data to be normally distributed,hence for a better accuracy skewness needs to be handled!

In [ ]:
data.skew()

Chlorides,residual sugar ,free sulfur dioxide,total sulfur dioxide,sulphates are skewed!


In [ ]:
from scipy.stats import boxcox

In [ ]:
non_skew,_=boxcox(data["chlorides"])
pd.Series(non_skew).skew()


# Data_preprocessing

Handling skewed data!(log transfomations,sqrt,cbrt,boxcox reduce the skewness)
whichever transformation gives better accuracy can be used!

In [ ]:
data["chlorides"],_=boxcox(data["chlorides"])
data["sulphates"]=np.log(data["sulphates"])
data["free sulfur dioxide"]=np.log(data["free sulfur dioxide"])
data["total sulfur dioxide"],_=boxcox(data["total sulfur dioxide"])
data["residual sugar"],_=boxcox(data["residual sugar"])

# Handling outliers! 

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

A custom transfomer to handle outliers! returns the dataset with outliers removed!

In [ ]:
class outlir(BaseEstimator,TransformerMixin):
    def __init__(self,k):
        self.k=k
    def fit(self,X,y=None):
        return self
    def transform(self,X):
        lof=LocalOutlierFactor(self.k)
        return X.iloc[lof.fit_predict(X)!=-1,:]

In [ ]:
ol=outlir(20)
data=ol.fit_transform(data)
data

# splitting the set!


Since the o/p is skewed(unequal no of zeros and ones)splitting must me stratified!

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
split = StratifiedShuffleSplit( test_size=0.2, random_state=42)

In [ ]:
for train_index,test_index in split.split(data,data["quality_mark"]):
    strat_train_set = data.iloc[train_index,:]
    strat_test_set = data.iloc[test_index,:]

In [ ]:
X_train=strat_train_set.iloc[:,:-1]
y_train=strat_train_set.iloc[:,-1]
X_test=strat_test_set.iloc[:,:-1]
y_test=strat_test_set.iloc[:,-1]

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

# scaling of data

In [ ]:
sc=StandardScaler()


In [ ]:
X_t=sc.fit_transform(X_train)

In [ ]:
X_test_scaled=sc.transform(X_test)

# MODELS


**RFC**

In [ ]:
rfc=RandomForestClassifier()

In [ ]:
rfc.fit(X_t,y_train)

In [ ]:
cross_val_score(rfc,X_train,y_train,cv=10,scoring="accuracy").mean()

In [ ]:
accuracy_score(y_test,rfc.predict(X_test_scaled))


In [ ]:
print(classification_report(y_test,rfc.predict(X_test_scaled)))

***Random Forest:95% accuracy!!:))*****

**LOGISTIC REGRESSION**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr=LogisticRegression()


In [ ]:
lr.fit(X_t,y_train)

In [ ]:
cross_val_score(lr,X_t,y_train,cv=100,scoring="accuracy").mean()

In [ ]:
accuracy_score(lr.predict(X_test_scaled),y_test)

In [ ]:
print(classification_report(y_test,lr.predict(X_test_scaled)))

***Logistic Regression:90% accuracy***

***Support vector classfier***

In [ ]:
from sklearn.svm import SVC

In [ ]:
supc=SVC()
supc.fit(X_t,y_train)

In [ ]:
cross_val_score(supc,X_t,y_train,cv=10,scoring="accuracy").mean()

In [ ]:
accuracy_score(supc.predict(X_test_scaled),y_test)

In [ ]:
print(classification_report(y_test,supc.predict(X_test_scaled)))

***90% accuracy***

# Random forests works good!moreover, on using GridsearchCV, and tuning the hyperparameters, 96~97% of accuracy can be achieved

**Thankyou for going through this notebook!if there are any errors please do correct me,im a beginner!**   **:D**